In [1]:
# Open Database

import pandas as pd

from sqlalchemy import create_engine

sqlitedb_abs_path = '/vagrant/Data/raw_data_project_m1.db'

conn_str = f'sqlite:///{sqlitedb_abs_path}'

engine = create_engine(conn_str)

pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", engine)

#Call all tables

,name
0,personal_info
1,country_info
2,career_info
3,poll_info


In [2]:
# Create dataframe from db query to get uuid, Gender and Age

working_dataframe = pd.read_sql_query('''SELECT career_info.normalized_job_code, personal_info.gender, personal_info.age, country_info.country_code
FROM personal_info
JOIN career_info ON career_info.uuid = personal_info.uuid 
JOIN country_info ON personal_info.uuid = country_info.uuid ;  ''', engine)

working_dataframe

,normalized_job_code,gender,age,country_code
0,None,male,61 years old,AT
1,861a9b9151e11362eb3c77ca914172d0,male,57 years old,AT
2,None,male,32 years old,AT
3,049a3f3a2b5f85cb2971ba77ad66e10c,Male,45 years old,AT
4,f4b2fb1aa40f661488e2782b6d57ad2f,Fem,41 years old,AT
...,...,...,...,...
9644,847165cfda6b1dc82ae22b967da8af2f,FeMale,37 years old,SK
9645,a4d5b8b38f9513825d0d94a981ebe962,Male,53 years old,SK
9646,None,male,1992,SK
9647,775190277a849cba701b306a7b374c0a,male,47 years old,SK


In [3]:
## Clean up gender

gender = working_dataframe['gender'].to_list()
clean_gender = []

for item in gender:
    if "fem" in item.lower():
        clean_gender.append("Female")
    else:
        clean_gender.append("Male")
        
working_dataframe['gender'] = clean_gender

working_dataframe

,normalized_job_code,gender,age,country_code
0,None,Male,61 years old,AT
1,861a9b9151e11362eb3c77ca914172d0,Male,57 years old,AT
2,None,Male,32 years old,AT
3,049a3f3a2b5f85cb2971ba77ad66e10c,Male,45 years old,AT
4,f4b2fb1aa40f661488e2782b6d57ad2f,Female,41 years old,AT
...,...,...,...,...
9644,847165cfda6b1dc82ae22b967da8af2f,Female,37 years old,SK
9645,a4d5b8b38f9513825d0d94a981ebe962,Male,53 years old,SK
9646,None,Male,1992,SK
9647,775190277a849cba701b306a7b374c0a,Male,47 years old,SK


In [4]:
# cleaning age

age = working_dataframe['age'].to_list()
clean_age = []

for item in age:
    if 'old' in item:
        clean_age.append(int(item[0:2]) + 4) 
    else:
        clean_age.append(2020 - int(item) + 4)

working_dataframe['age'] = clean_age

working_dataframe

,normalized_job_code,gender,age,country_code
0,None,Male,65,AT
1,861a9b9151e11362eb3c77ca914172d0,Male,61,AT
2,None,Male,36,AT
3,049a3f3a2b5f85cb2971ba77ad66e10c,Male,49,AT
4,f4b2fb1aa40f661488e2782b6d57ad2f,Female,45,AT
...,...,...,...,...
9644,847165cfda6b1dc82ae22b967da8af2f,Female,41,SK
9645,a4d5b8b38f9513825d0d94a981ebe962,Male,57,SK
9646,None,Male,32,SK
9647,775190277a849cba701b306a7b374c0a,Male,51,SK


In [5]:
#retrieving job code

import requests
import json

#first get unique values
unique_values_job = working_dataframe['normalized_job_code'].unique()


#getting job data from Api
jobs_id = []
counter = 0
for code in unique_values_job:
    response = requests.get(f'http://api.dataatwork.org/v1/jobs/{code}')
    jobs_id.append(response.json())
    counter +=1
    if counter % 10 == 0:
        print(counter)

#Create Dataframe from results
job_title_df = pd.DataFrame.from_records(jobs_id[1:]).drop(columns=['normalized_job_title', 'parent_uuid'])

job_title_df





10
20
30
40
50
60
70
80
90
100
110
120
130
140
150


,uuid,title
0,861a9b9151e11362eb3c77ca914172d0,Automatic Data Processing Planner
1,049a3f3a2b5f85cb2971ba77ad66e10c,Data Coordinator
2,f4b2fb1aa40f661488e2782b6d57ad2f,Database Developer
3,27af8700f5577cec835acee2cb90a2ff,Data Entry Specialist
4,c1b670eba9ccb65e7c99f7da116d5b9c,Database Architect
...,...,...
151,b0fa6ede410f50b82ab74f5a705fe699,Analytical Data Miner
152,559a21f836c93876f31b60e6d10656a7,Data Analysis Assistant
153,c1fb1a01b78373ac2153c66fa08d16dc,Data Examination Clerk
154,05bb9a333a66d6eb151e253623efe1c0,Data Entry Clerk


In [6]:
# Get normalized_job_title list

normalized_job_codes = working_dataframe['normalized_job_code'].to_list()
available_job_codes = job_title_df.to_dict('records')
job_titles = []



# Add title to working dataframe by joining the uuids from both dataframes
for code in normalized_job_codes:
    # Query the job title DF so it returns the uuid for the current row
    # normalized job code in the working dataframe
    if code == None:
        job_titles.append(None)
    else:
        for row in available_job_codes:
            if row['uuid'] == code:
                job_titles.append(row['title'])

# Add title column to the working dataframe
working_dataframe['title'] = job_titles

working_dataframe
        
 

    

,normalized_job_code,gender,age,country_code,title
0,None,Male,65,AT,None
1,861a9b9151e11362eb3c77ca914172d0,Male,61,AT,Automatic Data Processing Planner
2,None,Male,36,AT,None
3,049a3f3a2b5f85cb2971ba77ad66e10c,Male,49,AT,Data Coordinator
4,f4b2fb1aa40f661488e2782b6d57ad2f,Female,45,AT,Database Developer
...,...,...,...,...,...
9644,847165cfda6b1dc82ae22b967da8af2f,Female,41,SK,Data Warehouse Developer
9645,a4d5b8b38f9513825d0d94a981ebe962,Male,57,SK,Database Manager
9646,None,Male,32,SK,None
9647,775190277a849cba701b306a7b374c0a,Male,51,SK,Data Officer


In [7]:
# Now web scraping to get the countries.
# First, install the library Beautiful Soup

!pip install BeautifulSoup4


     |████████████████████████████████| 115 kB 16.8 MB/s eta 0:00:01


In [264]:
import requests

url = 'https://ec.europa.eu/eurostat/statistics-explained/index.php/Glossary:Country_codes'
html = requests.get(url).content
html[:5000]

b'\xef\xbb\xbf<!DOCTYPE html>\n<html lang="en" dir="ltr" class="client-nojs">\n<head>\n<meta http-equiv="X-UA-Compatible" content="Ie=Edge,chrome=1"/> <meta charset="UTF-8" /><title>Glossary:Country codes - Statistics Explained</title>\n<meta name="generator" content="MediaWiki 1.22.5" />\n<link rel="shortcut icon" href="https://ec.europa.eu/eurostat/estat-theme/images/favicon.ico" />\n<link rel="search" type="application/opensearchdescription+xml" href="/eurostat/statistics-explained/opensearch_desc.php" title="Statistics Explained (en)" />\n<link rel="EditURI" type="application/rsd+xml" href="https://ec.europa.eu/eurostat/statistics-explained/api.php?action=rsd" />\n<link rel="alternate" type="application/atom+xml" title="Statistics Explained Atom feed" href="/eurostat/statistics-explained/index.php?title=Special:RecentChanges&amp;feed=atom" />\n<link rel="stylesheet" href="https://ec.europa.eu/eurostat/statistics-explained/load.php?debug=true&amp;lang=en&amp;modules=ext.rtlcite&amp;

In [265]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html, 'html.parser')

soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta content="Ie=Edge,chrome=1" http-equiv="X-UA-Compatible"/> <meta charset="utf-8"/><title>Glossary:Country codes - Statistics Explained</title>
<meta content="MediaWiki 1.22.5" name="generator"/>
<link href="https://ec.europa.eu/eurostat/estat-theme/images/favicon.ico" rel="shortcut icon"/>
<link href="/eurostat/statistics-explained/opensearch_desc.php" rel="search" title="Statistics Explained (en)" type="application/opensearchdescription+xml"/>
<link href="https://ec.europa.eu/eurostat/statistics-explained/api.php?action=rsd" rel="EditURI" type="application/rsd+xml"/>
<link href="/eurostat/statistics-explained/index.php?title=Special:RecentChanges&amp;feed=atom" rel="alternate" title="Statistics Explained Atom feed" type="application/atom+xml"/>
<link href="https://ec.europa.eu/eurostat/statistics-explained/load.php?debug=true&amp;lang=en&amp;modules=ext.rtlcite&amp;only=styles&amp;skin=statexpflat&amp;*" rel="

In [266]:
my_list_tables = []

for table in all_tables:
    table_rows = table.find_all('tr')
    for tr in table_rows:
        td = tr.find_all('td')
        for tr in td:
            my_list_tables.append(tr.text)
        

my_list_tables

['Belgium\n',
 '(BE)\n',
 'Greece\n',
 '(EL)\n',
 'Lithuania\n',
 '(LT)\n',
 'Portugal\n',
 '(PT)\n',
 'Bulgaria\n',
 '(BG)\n',
 'Spain\n',
 '(ES)\n',
 'Luxembourg\n',
 '(LU)\n',
 'Romania\n',
 '(RO)\n',
 'Czechia\n',
 '(CZ)\n',
 'France\n',
 '(FR)\n',
 'Hungary\n',
 '(HU)\n',
 'Slovenia\n',
 '(SI)\n',
 'Denmark\n',
 '(DK)\n',
 'Croatia\n',
 '(HR)\n',
 'Malta\n',
 '(MT)\n',
 'Slovakia\n',
 '(SK)\n',
 'Germany\n',
 '(DE)\n',
 'Italy\n',
 '(IT)\n',
 'Netherlands\n',
 '(NL)\n',
 'Finland\n',
 '(FI)\n',
 'Estonia\n',
 '(EE)\n',
 'Cyprus\n',
 '(CY)\n',
 'Austria\n',
 '(AT)\n',
 'Sweden\n',
 '(SE)\n',
 'Ireland\n',
 '(IE)\n',
 'Latvia\n',
 '(LV)\n',
 'Poland\n',
 '(PL)\n',
 ' Iceland\n',
 ' (IS)\n',
 '\n',
 ' Norway\n',
 ' (NO)\n',
 ' Liechtenstein\n',
 ' (LI)\n',
 '\n',
 ' Switzerland\n',
 ' (CH)\n',
 ' United Kingdom\n',
 ' (UK)\n',
 '\n',
 ' Montenegro\n',
 ' (ME)\n',
 ' North Macedonia\n',
 ' (MK)\n',
 ' Albania\n',
 ' (AL)\n',
 'Serbia\n',
 ' (RS)\n',
 ' Turkey\n',
 ' (TR)\n',
 ' Bosnia

In [289]:
#Drop \n from my list_tables and cleaning

converted_list = []

for element in my_list_tables:
    converted_list.append(element.strip())

converted_list = list(filter(None, converted_list))

converted_list

['Belgium',
 '(BE)',
 'Greece',
 '(EL)',
 'Lithuania',
 '(LT)',
 'Portugal',
 '(PT)',
 'Bulgaria',
 '(BG)',
 'Spain',
 '(ES)',
 'Luxembourg',
 '(LU)',
 'Romania',
 '(RO)',
 'Czechia',
 '(CZ)',
 'France',
 '(FR)',
 'Hungary',
 '(HU)',
 'Slovenia',
 '(SI)',
 'Denmark',
 '(DK)',
 'Croatia',
 '(HR)',
 'Malta',
 '(MT)',
 'Slovakia',
 '(SK)',
 'Germany',
 '(DE)',
 'Italy',
 '(IT)',
 'Netherlands',
 '(NL)',
 'Finland',
 '(FI)',
 'Estonia',
 '(EE)',
 'Cyprus',
 '(CY)',
 'Austria',
 '(AT)',
 'Sweden',
 '(SE)',
 'Ireland',
 '(IE)',
 'Latvia',
 '(LV)',
 'Poland',
 '(PL)',
 'Iceland',
 '(IS)',
 'Norway',
 '(NO)',
 'Liechtenstein',
 '(LI)',
 'Switzerland',
 '(CH)',
 'United Kingdom',
 '(UK)',
 'Montenegro',
 '(ME)',
 'North Macedonia',
 '(MK)',
 'Albania',
 '(AL)',
 'Serbia',
 '(RS)',
 'Turkey',
 '(TR)',
 'Bosnia and Herzegovina',
 '(BA)',
 'Kosovo*',
 '(XK[1])',
 'Armenia',
 '(AM)',
 'Belarus',
 '(BY)',
 'Georgia',
 '(GE)',
 'Azerbaijan',
 '(AZ)',
 'Moldova',
 '(MD)',
 'Ukraine',
 '(UA)',
 'Algeri

In [301]:
# Converting list to dictionary

titles = []
codes = []

my_dict_tables = {"Country": titles, "Country_code": codes}

for item in converted_list:
    if "(" in item and " " not in item:
        codes.append(item)
    else:
        titles.append(item)

my_dict_tables['Country_code']
my_dict_tables['Country']


['Belgium',
 'Greece',
 'Lithuania',
 'Portugal',
 'Bulgaria',
 'Spain',
 'Luxembourg',
 'Romania',
 'Czechia',
 'France',
 'Hungary',
 'Slovenia',
 'Denmark',
 'Croatia',
 'Malta',
 'Slovakia',
 'Germany',
 'Italy',
 'Netherlands',
 'Finland',
 'Estonia',
 'Cyprus',
 'Austria',
 'Sweden',
 'Ireland',
 'Latvia',
 'Poland',
 'Iceland',
 'Norway',
 'Liechtenstein',
 'Switzerland',
 'United Kingdom',
 'Montenegro',
 'North Macedonia',
 'Albania',
 'Serbia',
 'Turkey',
 'Bosnia and Herzegovina',
 'Kosovo*',
 'Armenia',
 'Belarus',
 'Georgia',
 'Azerbaijan',
 'Moldova',
 'Ukraine',
 'Algeria',
 'Lebanon',
 'Syria',
 'Egypt',
 'Libya',
 'Tunisia',
 'Israel',
 'Morocco',
 'Jordan',
 'Palestine [2]',
 'Russia',
 'Argentina',
 'China (except Hong Kong)',
 'Mexico',
 'South Africa',
 'Australia',
 'Hong Kong',
 'Nigeria',
 'South Korea',
 'Brazil',
 'India',
 'New Zealand',
 'Taiwan',
 'Canada',
 'Japan',
 'Singapore',
 'United States']

In [300]:
# Converting dictionary to Dataframe

df_countries = pd.DataFrame(my_dict_tables)

df_countries.head()

,Country,Country_code
0,Belgium,(BE)
1,Greece,(EL)
2,Lithuania,(LT)
3,Portugal,(PT)
4,Bulgaria,(BG)
